1. Stock price prediction with any one meta-heuristic approach

In [35]:
import numpy as np
import random
import yfinance as yf
import time

# --- Load data ---
ticker = yf.Ticker("TSLA")
price_data = ticker.history(period="1d", interval="1m")
n = 250
recent = price_data.tail(n)
prices = recent['Close'].values

In [36]:
# --- Hyperparameters ---
window_size = 5
hidden_size = 10
param_size = window_size * hidden_size + hidden_size + hidden_size + 1

# --- Prepare dataset (X: past window_size prices, y: next price) ---
X = []
y = []
for i in range(len(prices) - window_size):
    X.append(prices[i:i+window_size])
    y.append(prices[i+window_size])
X = np.array(X)
y = np.array(y)

# --- Train/Test Split ---
split_idx = int(0.7 * len(X))
X_train = X[:split_idx]
y_train = y[:split_idx]
X_test = X[split_idx:]
y_test = y[split_idx:]

# --- Define MLP Model ---
def forward(x, params, input_dim, hidden_dim):
    W1 = params[:input_dim * hidden_dim].reshape(input_dim, hidden_dim)
    b1 = params[input_dim * hidden_dim : input_dim * hidden_dim + hidden_dim]
    W2 = params[input_dim * hidden_dim + hidden_dim : input_dim * hidden_dim + hidden_dim + hidden_dim].reshape(hidden_dim, 1)
    b2 = params[-1]

    hidden = np.dot(x, W1) + b1
    hidden = np.maximum(hidden, 0)  # ReLU
    output = np.dot(hidden, W2) + b2
    return output.squeeze()

# --- Loss Function (MSE) ---
def mse_loss(params, X, y, input_dim, hidden_dim):
    preds = np.array([forward(x, params, input_dim, hidden_dim) for x in X])
    return np.mean((preds - y)**2)

# --- Particle Class ---
class Particle:
    def __init__(self, dim, input_dim, hidden_dim, X, y):
        self.position = np.random.randn(dim)
        self.velocity = np.random.randn(dim) * 0.1
        self.pbest_position = self.position.copy()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.X = X
        self.y = y
        self.pbest_score = self.evaluate(self.position)

    def evaluate(self, position):
        return mse_loss(position, self.X, self.y, self.input_dim, self.hidden_dim)

    def update_pbest(self):
        score = self.evaluate(self.position)
        if score < self.pbest_score:
            self.pbest_score = score
            self.pbest_position = self.position.copy()

# --- PSO Class ---
class PSO:
    def __init__(self, n_particles, dim, X, y, n_iterations, input_dim, hidden_dim):
        self.n_particles = n_particles
        self.dim = dim
        self.X = X
        self.y = y
        self.n_iterations = n_iterations
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.particles = [Particle(dim, input_dim, hidden_dim, X, y) for _ in range(n_particles)]
        self.gbest_position = None
        self.gbest_score = float('inf')

    def optimize(self):
        for particle in self.particles:
            if particle.pbest_score < self.gbest_score:
                self.gbest_score = particle.pbest_score
                self.gbest_position = particle.pbest_position.copy()

        for it in range(self.n_iterations):
            for particle in self.particles:
                # Update velocity
                inertia = 0.7
                cognitive_coeff = 1.5
                social_coeff = 1.5
                r1 = np.random.rand(self.dim)
                r2 = np.random.rand(self.dim)

                cognitive = cognitive_coeff * r1 * (particle.pbest_position - particle.position)
                social = social_coeff * r2 * (self.gbest_position - particle.position)
                particle.velocity = inertia * particle.velocity + cognitive + social

                # Update position
                particle.position += particle.velocity

                # Update pbest
                particle.update_pbest()

                # Update gbest
                if particle.pbest_score < self.gbest_score:
                    self.gbest_score = particle.pbest_score
                    self.gbest_position = particle.pbest_position.copy()

            if it % 10 == 0:
                print(f"Iteration {it}, Best Loss: {self.gbest_score:.6f}")

        return self.gbest_position

# --- Run PSO ---
pso = PSO(n_particles=30, dim=param_size, X=X_train, y=y_train, n_iterations=100, input_dim=window_size, hidden_dim=hidden_size)
best_params = pso.optimize()

# --- Test Evaluation with timestamp ---
predictions = np.array([forward(x, best_params, window_size, hidden_size) for x in X_test])

test_timestamps = recent.index[split_idx+window_size:]

print(f"{'Timestamp':<25} {'Real':<15} {'Prediction':<15}")
print("-"*55)
for ts, real, pred in zip(test_timestamps, y_test, predictions):
    print(f"{ts} {real:<15.6f} {pred:<15.6f}")

mse_test = np.mean((predictions - y_test)**2)
absolute_errors = np.abs(predictions - y_test)
mean_absolute_error = np.mean(absolute_errors)

print("\nTest MSE:", round(mse_test, 6))
print("Test Mean Absolute Error:", round(mean_absolute_error, 6))

Iteration 0, Best Loss: 10.421326
Iteration 10, Best Loss: 7.030207
Iteration 20, Best Loss: 5.906097
Iteration 30, Best Loss: 0.874574
Iteration 40, Best Loss: 0.874574
Iteration 50, Best Loss: 0.874574
Iteration 60, Best Loss: 0.347195
Iteration 70, Best Loss: 0.347195
Iteration 80, Best Loss: 0.347195
Iteration 90, Best Loss: 0.347195
Timestamp                 Real            Prediction     
-------------------------------------------------------
2025-04-25 14:46:00-04:00 285.769989      285.445624     
2025-04-25 14:47:00-04:00 285.709015      285.551159     
2025-04-25 14:48:00-04:00 285.890015      285.634423     
2025-04-25 14:49:00-04:00 285.829895      285.650232     
2025-04-25 14:50:00-04:00 285.591888      285.677225     
2025-04-25 14:51:00-04:00 285.494995      285.646279     
2025-04-25 14:52:00-04:00 284.730011      285.562372     
2025-04-25 14:53:00-04:00 285.214996      285.477589     
2025-04-25 14:54:00-04:00 285.570007      285.282842     
2025-04-25 14:55:00-04:0

2. Shortest Distance Calculation by meta-heuristic approach